## 2.4 Model Selection 소개
### 학습/테스트 데이터 셋 분리 – train_test_split()

In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
dt_clf = DecisionTreeClassifier()
train_data = iris.data
train_label = iris.target
dt_clf.fit(train_data, train_label)

# 학습 데이터 셋으로 예측 수행
pred = dt_clf.predict(train_data)
print('예측 정확도:',accuracy_score(train_label,pred))

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

dt_clf = DecisionTreeClassifier( )
iris_data = load_iris()

X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.3, random_state=121)





In [ ]:
dt_clf.fit(X_train, y_train)
pred = dt_clf.predict(X_test)
print('예측 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))

### 교차 검증
* K 폴드 

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
import numpy as np

iris = load_iris()
features = iris.data
label = iris.target
dt_clf = DecisionTreeClassifier(random_state=156)

# 5개의 폴드 세트로 분리하는 KFold 객체와 폴드 세트별 정확도를 담을 리스트 객체 생성.
kfold = KFold(n_splits=5)
cv_accuracy = []
print('붓꽃 데이터 세트 크기:',features.shape[0])


In [ ]:
n_iter = 0

# KFold객체의 split( ) 호출하면 폴드 별 학습용, 검증용 테스트의 로우 인덱스를 array로 반환  
for train_index, test_index  in kfold.split(features):
    # kfold.split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)
    n_iter += 1
    # 반복 시 마다 정확도 측정 
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 개별 iteration별 정확도를 합하여 평균 정확도 계산 
print('\n## 평균 검증 정확도:', np.mean(cv_accuracy)) 

* Stratified K 폴드

In [ ]:
import pandas as pd

iris = load_iris()

iris_df = pd.DataFrame(data=iris.data, columns=iris.feature_names)
iris_df['label']=iris.target
iris_df['label'].value_counts()



In [ ]:
kfold = KFold(n_splits=3)
# kfold.split(X)는 폴드 세트를 3번 반복할 때마다 달라지는 학습/테스트 용 데이터 로우 인덱스 번호 반환. 
n_iter =0
for train_index, test_index  in kfold.split(iris_df):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())
    

In [ ]:
from sklearn.model_selection import StratifiedKFold

skf = StratifiedKFold(n_splits=3)
n_iter=0

for train_index, test_index in skf.split(iris_df, iris_df['label']):
    n_iter += 1
    label_train= iris_df['label'].iloc[train_index]
    label_test= iris_df['label'].iloc[test_index]
    print('## 교차 검증: {0}'.format(n_iter))
    print('학습 레이블 데이터 분포:\n', label_train.value_counts())
    print('검증 레이블 데이터 분포:\n', label_test.value_counts())

In [ ]:
dt_clf = DecisionTreeClassifier(random_state=156)

skfold = StratifiedKFold(n_splits=3)
n_iter=0
cv_accuracy=[]

# StratifiedKFold의 split( ) 호출시 반드시 레이블 데이터 셋도 추가 입력 필요  
for train_index, test_index  in skfold.split(features, label):
    # split( )으로 반환된 인덱스를 이용하여 학습용, 검증용 테스트 데이터 추출
    X_train, X_test = features[train_index], features[test_index]
    y_train, y_test = label[train_index], label[test_index]
    #학습 및 예측 
    dt_clf.fit(X_train , y_train)    
    pred = dt_clf.predict(X_test)

    # 반복 시 마다 정확도 측정 
    n_iter += 1
    accuracy = np.round(accuracy_score(y_test,pred), 4)
    train_size = X_train.shape[0]
    test_size = X_test.shape[0]
    print('\n#{0} 교차 검증 정확도 :{1}, 학습 데이터 크기: {2}, 검증 데이터 크기: {3}'
          .format(n_iter, accuracy, train_size, test_size))
    print('#{0} 검증 세트 인덱스:{1}'.format(n_iter,test_index))
    cv_accuracy.append(accuracy)
    
# 교차 검증별 정확도 및 평균 정확도 계산 
print('\n## 교차 검증별 정확도:', np.round(cv_accuracy, 4))
print('## 평균 검증 정확도:', np.round(np.mean(cv_accuracy), 4))

* cross_val_score( )

In [8]:
import numpy as np
import pandas as pd
# 트리 계열 분류 알고리즘
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score, cross_validate
# 데이터 : 아이리스 데이터 셋 로딩 : load_iris() return Dict 
from sklearn.datasets import load_iris

# 데이터 로딩
iris_data = load_iris() # Diction key data, target
# iris_data.data[:5], iris_data.target[:5]
# (array([[5.1, 3.5, 1.4, 0.2],
#         [4.9, 3. , 1.4, 0.2],
#         [4.7, 3.2, 1.3, 0.2],
#         [4.6, 3.1, 1.5, 0.2],
#         [5. , 3.6, 1.4, 0.2]]),
#  array([0, 0, 0, 0, 0]))
data = iris_data.data # 데이터 추출
label = iris_data.target # 답 추출

# DT object 
dt_clf = DecisionTreeClassifier(random_state=42) # 데이터 추출을 고정시키는 역할

# cross_val_score(cv=3) => [acc score,acc score,acc score] 
scores = cross_val_score(dt_clf, data, label, scoring='accuracy', cv=3, verbose=2)
print(scores) # 리스트 출력
print(np.mean(scores)) # 평균 출력



[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[CV] END .................................................... total time=   0.0s
[0.98 0.94 0.96]
0.96


In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import cross_val_score , cross_validate
from sklearn.datasets import load_iris

iris_data = load_iris()
dt_clf = DecisionTreeClassifier(random_state=156)

data = iris_data.data
label = iris_data.target

# 성능 지표는 정확도(accuracy) , 교차 검증 세트는 3개 
scores = cross_val_score(dt_clf , data , label , scoring='accuracy',cv=3)
print('교차 검증별 정확도:',np.round(scores, 4))
print('평균 검증 정확도:', np.round(np.mean(scores), 4))

* GridSearchCV

In [12]:
import numpy as np
import pandas as pd

from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split

# 데이터 로딩, 학습데이터/테스트데이터 분리
iris_data = load_iris()
# 학습    테스트   합습답   테스트답
X_train, X_test, y_train, y_test = train_test_split(
      iris_data.data # 전체 data
    , iris_data.target # 전체 데이터 답 label
    , test_size= 0.2 # 테스트데이터 크기 지정 20%
    , random_state=42 # seed 값 고정 
)

# DT object
dt_clf = DecisionTreeClassifier() # 오브젝트 생성

# 그리드에서 하이퍼 파라미터 설정
grid_param = {
      'max_depth' : [1,2,3]
    , 'min_samples_split' : [2,3]
} 

# 그리드 오브젝트 생성
grid_dt_clf = GridSearchCV(
      dt_clf # Es
    , param_grid= grid_param
    , cv=3
    , refit=True
)
# 알고리즘 학습 : 학습데이터,학습데이터의 답
grid_dt_clf.fit(X_train, y_train) # 학습되면 모델이 생성

# 결과를 저장하고 있는 속성 : cv_results_
scores_df = pd.DataFrame(grid_dt_clf.cv_results_)
scores_df

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_max_depth,param_min_samples_split,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score
0,0.000332,0.000470,0.000340,0.000481,1,2,"{'max_depth': 1, 'min_samples_split': 2}",0.675,0.675,0.675,0.675000,1.110223e-16,5
1,0.000330,0.000467,0.000000,0.000000,1,3,"{'max_depth': 1, 'min_samples_split': 3}",0.675,0.675,0.675,0.675000,1.110223e-16,5
2,0.000366,0.000371,0.000526,0.000744,2,2,"{'max_depth': 2, 'min_samples_split': 2}",0.975,0.900,0.950,0.941667,3.118048e-02,1
3,0.000329,0.000466,0.000334,0.000472,2,3,"{'max_depth': 2, 'min_samples_split': 3}",0.975,0.900,0.950,0.941667,3.118048e-02,1
4,0.000357,0.000505,0.000000,0.000000,3,2,"{'max_depth': 3, 'min_samples_split': 2}",0.950,0.900,0.950,0.933333,2.357023e-02,3
5,0.000307,0.000434,0.000329,0.000466,3,3,"{'max_depth': 3, 'min_samples_split': 3}",0.950,0.900,0.950,0.933333,2.357023e-02,3


In [13]:
# 최적 하이퍼 파라미터 추출
grid_dt_clf.best_params_

{'max_depth': 2, 'min_samples_split': 2}

In [14]:
# 최고 점수
grid_dt_clf.best_score_

0.9416666666666668

In [15]:
# 최적 모델 추출 : 모델 => 학습이 완료된 것
from sklearn.metrics import accuracy_score
estimator = grid_dt_clf.best_estimator_
pred = estimator.predict(X_test) # 테스트데이터로 예측값 추출

# 테스트데이터(New Data)의 성능 : 일반화 성능, 진짜 성능
print(accuracy_score(y_test, pred))

0.9666666666666667


In [ ]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV

# 데이터를 로딩하고 학습데이타와 테스트 데이터 분리
iris = load_iris()
X_train, X_test, y_train, y_test = train_test_split(iris_data.data, iris_data.target, 
                                                    test_size=0.2, random_state=121)
dtree = DecisionTreeClassifier()

### parameter 들을 dictionary 형태로 설정
parameters = {'max_depth':[1,2,3], 'min_samples_split':[2,3]}

In [ ]:
import pandas as pd

# param_grid의 하이퍼 파라미터들을 3개의 train, test set fold 로 나누어서 테스트 수행 설정.  
### refit=True 가 default 임. True이면 가장 좋은 파라미터 설정으로 재 학습 시킴.  
grid_dtree = GridSearchCV(dtree, param_grid=parameters, cv=3, refit=True)

# 붓꽃 Train 데이터로 param_grid의 하이퍼 파라미터들을 순차적으로 학습/평가 .
grid_dtree.fit(X_train, y_train)

# GridSearchCV 결과 추출하여 DataFrame으로 변환
scores_df = pd.DataFrame(grid_dtree.cv_results_)
scores_df[['params', 'mean_test_score', 'rank_test_score', \
           'split0_test_score', 'split1_test_score', 'split2_test_score']]

In [ ]:
print('GridSearchCV 최적 파라미터:', grid_dtree.best_params_)
print('GridSearchCV 최고 정확도: {0:.4f}'.format(grid_dtree.best_score_))

In [ ]:
# GridSearchCV의 refit으로 이미 학습이 된 estimator 반환
estimator = grid_dtree.best_estimator_

# GridSearchCV의 best_estimator_는 이미 최적 하이퍼 파라미터로 학습이 됨
pred = estimator.predict(X_test)
print('테스트 데이터 세트 정확도: {0:.4f}'.format(accuracy_score(y_test,pred)))